In [7]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [8]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [9]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCNO', 'des_col': 'scno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTRSTOCK_ID', 'des_col': 'mtrstock_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_LOC', 'des_col': 'meter_loc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_HEIGHT', 'des_col': 'meter_height', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MULTPF', 'des_col': 'multpf', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_SITE_ADDRESS_1', 'des_col': 'meter_site_address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_SITE_ADDRESS_2', 'des_col': 'meter_site_address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_SITE_ADDRESS_3', 'des_col': 'meter_site_address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_SITE_ADDRESS_4', 'des_col': 'meter_site_address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTROPNRDG', 'des_col': 'mtropnrdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'integer'},
    #{'src_col': 'MTROPNRDGDT', 'des_col': 'mtropnrdgdt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'}, wrong record in this field 
    {'src_col': 'MTRCURRENTSTAT', 'des_col': 'mtrcurrentstat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COVER_SEALNO_1', 'des_col': 'cover_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COVER_SEALNO_2', 'des_col': 'cover_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COVER_SEALNO_3', 'des_col': 'cover_sealno_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COVER_SEALNO_4', 'des_col': 'cover_sealno_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TERMINAL_SEALNO_1', 'des_col': 'terminal_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TERMINAL_SEALNO_2', 'des_col': 'terminal_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'KWH_STATUS', 'des_col': 'kwh_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'KVA_STATUS', 'des_col': 'kva_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'KVAH_STATUS', 'des_col': 'kvah_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'KWH_RDG', 'des_col': 'kwh_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'KVA_RDG', 'des_col': 'kva_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'KVAH_RDG', 'des_col': 'kvah_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KWH', 'des_col': 'mf_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KVA', 'des_col': 'mf_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MF_KVAH', 'des_col': 'mf_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KWH', 'des_col': 'df_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KVA', 'des_col': 'df_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DF_KVAH', 'des_col': 'df_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CAPACITOR_REQUIRED', 'des_col': 'capacitor_required', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CAPACITOR_FIXED', 'des_col': 'capacitor_fixed', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_BOX', 'des_col': 'meter_box', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CAPACITOR_SLNO', 'des_col': 'capacitor_slno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CAPACITOR_MAKE', 'des_col': 'capacitor_make', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RATEDCTRATIO_N', 'des_col': 'ratedctratio_n', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'RATEDPTRATIO_N', 'des_col': 'ratedptratio_n', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'RATEDCTRATIO_D', 'des_col': 'ratedctratio_d', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'RATEDPTRATIO_D', 'des_col': 'ratedptratio_d', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},    
    {'src_col': 'TERMINAL_SEALNO_3', 'des_col': 'terminal_sealno_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COVER_SEALNO_5', 'des_col': 'cover_sealno_5', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.MT_CONS_MTR_REL', 'public', 'mt_cons_mtr_rel', cols_map)
print("Total Migrated Records (mt_cons_mtr_rel): " + str(migrated_row_count))


Total Migrated Records (mt_cons_mtr_rel): Total Src Records: 1498106 Total migrated Records to dest: 1498106
